# Skript4: Finetuning eines Multilanguage BERT-Modells für NER anhand von GermEval2014

Dieses Notebook beschreibt das Training eines Transformer Modells für die Task des Named Entity Recognition. Es wird ein BERT-Modell mit der nativen Hugging Face Transformers Bibliothek trainiert. Das Training erfolgt gemäß des Splits wie es die Shared Task 2014 vorgab.

In [ ]:
# Installation der benötigten Bibliotheken
#!pip install transformers
#!pip install datasets

Zunächst werden die Skriptparameter gesetzt. Die `task` Variable gibt an, für welche Aufgabe das Modell trainiert werden soll.
`ner` steht hier für Named Entity Recognition, dabei handelt es sich um ein Token-Klassifizierungsproblem. Die Variable `model_checkpoint` beinhaltet den Namen des zu nutzenden vortrainierten Transformer Modells. Das Modell `bert-base-multilingual-cased` ist ein BERT-Modell, welches mithilfe von Texten in 104 unterschiedlichen Sprachen trainiert wurde. Der Name kann durch einen beliebigen Modellcheckpoint aus dem Transformers Model Hub ersetzt werden:  https://huggingface.co/models

In [2]:
task = "ner"
model_checkpoint = 'bert-base-multilingual-cased'
batch_size=32

## Herunterladen des Datasets germeval2014

Neben der Transformers Bibliothek bietet die Hugging Face Inc. mit der `Datasets` Bibliothek eine Sammlung von Datensätzen und Metriken zum herunterladen an. Um diese zu nutzen, werden zuerst die beiden Methoden `load_dataset` und `load_metric` importiert.

In [3]:
from datasets import load_dataset, load_metric

Ebenso wie es für die vortrainierten Modelle einen Hub gibt, gibt es einen Hub für die verfügbaren Datasets: https://huggingface.co/datasets/germeval_14. Mit der Methode `load_dataset` kann das Dataset anschließend heruntergeladen werden.

In [4]:
datasets = load_dataset("germeval_14")

Dataset germ_eval14 downloaded and prepared to /root/.cache/huggingface/datasets/germ_eval14/germeval_14/2.0.0/2a7a0c62dc3278203778c3a16bfbe257d5656aa0f4ad1e84f357f4caa904e0da. Subsequent calls will reuse this data.


#### Exploration des Aufbaus der Daten

Das heruntergeladene Dataset ist ein `DatasetDcit-Objekt`, welches die Keys `train, validation` und `test` besitzt. Der Value für jeden Key ist das `Dataset-Objekt`, welches die jeweiligen Daten für den Split enthält.

Im Falle des GermEval2014 Datasets bestehen die Daten aus insgesamt 31300 Datensätzen mit den Features `id, source, tokens, ner_tags` und `nested_ner_tags`. Die `nested_ner_tags` finden im Rahmen dieses Anwendungsbeispiels keine Verwendung.

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'tokens', 'ner_tags', 'nested_ner_tags'],
        num_rows: 24000
    })
    validation: Dataset({
        features: ['id', 'source', 'tokens', 'ner_tags', 'nested_ner_tags'],
        num_rows: 2200
    })
    test: Dataset({
        features: ['id', 'source', 'tokens', 'ner_tags', 'nested_ner_tags'],
        num_rows: 5100
    })
})

Möchte man auf ein einzelnes Item aus dem Datensatz zugreifen, so wählt man den Key des Splits aus wählt anschließend den Index des gewünschten Items aus:

In [6]:
print(datasets["train"][0])

{'id': '0', 'ner_tags': [19, 0, 0, 0, 7, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'nested_ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'source': 'n-tv.de vom 26.02.2005 [2005-02-26] ', 'tokens': ['Schartau', 'sagte', 'dem', '"', 'Tagesspiegel', '"', 'vom', 'Freitag', ',', 'Fischer', 'sei', '"', 'in', 'einer', 'Weise', 'aufgetreten', ',', 'die', 'alles', 'andere', 'als', 'überzeugend', 'war', '"', '.']}


Die NER Tags sind bereits Integer ID codiert: 

In [7]:
datasets["train"][0]['ner_tags']

[19, 0, 0, 0, 7, 0, 0, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Das zugehörige Textlabel findet man unter dem `features`-Attribut des Datasets. Das Dataset GermEval2014 besitzt insgesamt 25 verschiedene Klassen:

In [9]:
datasets["train"].features["ner_tags"]

Sequence(feature=ClassLabel(num_classes=25, names=['O', 'B-LOC', 'I-LOC', 'B-LOCderiv', 'I-LOCderiv', 'B-LOCpart', 'I-LOCpart', 'B-ORG', 'I-ORG', 'B-ORGderiv', 'I-ORGderiv', 'B-ORGpart', 'I-ORGpart', 'B-OTH', 'I-OTH', 'B-OTHderiv', 'I-OTHderiv', 'B-OTHpart', 'I-OTHpart', 'B-PER', 'I-PER', 'B-PERderiv', 'I-PERderiv', 'B-PERpart', 'I-PERpart'], names_file=None, id=None), length=-1, id=None)

Die Klassenliste der Label kann wie folgt extrahiert werden:

In [10]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-LOC',
 'I-LOC',
 'B-LOCderiv',
 'I-LOCderiv',
 'B-LOCpart',
 'I-LOCpart',
 'B-ORG',
 'I-ORG',
 'B-ORGderiv',
 'I-ORGderiv',
 'B-ORGpart',
 'I-ORGpart',
 'B-OTH',
 'I-OTH',
 'B-OTHderiv',
 'I-OTHderiv',
 'B-OTHpart',
 'I-OTHpart',
 'B-PER',
 'I-PER',
 'B-PERderiv',
 'I-PERderiv',
 'B-PERpart',
 'I-PERpart']

#### Visualisierung von zufälligen Beispieldaten aus dem Dataset 
Die folgende Funktion wählt zufällig `N` Items aus dem übergebenen Dataset aus und gibt diese in einem `Pandas DataFrame` aus. Dabei werden die `ner_tags` in ihre entsprechenden Textlabel dekodiert.

In [11]:
# Quelle in Anlehnung an: https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb#scrollTo=X6HrpprwIrIz
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10,seed=None):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    random.seed(seed)
    #Befüllen mit Random indexes
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            random.seed(seed)
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
       # print(column)
       # print(typ)
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
            #die zeile hier unten ist aktiv
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

Die Methode kann nun dazu genutzt werden, eine tabellarische Ausgabe von Beispieldaten aus dem Dataset zu erzeugen. Dafür übergibt man das `datasets` Objekt und gibt den gewünschten Split an:

In [12]:
show_random_elements(datasets["train"], seed = 43)

,id,ner_tags,nested_ner_tags,source,tokens
0,1263,"[O, O, O, O, O, O, B-LOC, O, O]","[O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Khangchenne [2009-12-25],"[Er, hielt, sich, lieber, im, fernen, Ngari, auf, .]"
1,9374,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://www.verbaende.com/News.php4?m=56415 [2008-09-25],"[Daher, lässt, er, keine, Gelegenheit, aus, ,, die, Politik, wachzurütteln, und, zum, Handeln, zu, bewegen, .]"
2,22813,"[B-PER, O, O, O, O, O, O, O, O, B-LOC, O]","[O, O, O, O, O, O, O, O, O, O, O]",http://www.hellwegeranzeiger.de/afp/journal/doc/zuma-anc.htm [2007-12-16],"[Zuma, ist, der, Liebling, der, Armen, und, Benachteiligten, in, Südafrika, .]"
3,4716,"[O, O, O, O, O, O, O, O, B-ORGpart, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://www.handelsblatt.com/unternehmen/handel-dienstleister/lufthansa-fuerchtet-deutlichen-gewinnrueckgang;2251398 [2009-04-24],"[Außer, in, der, Cargo-Sparte, stehen, auch, an, dezentralen, Lufthansa-Standorten, des, Passagiergeschäfts, die, Zeichen, auf, Kurzarbeit, .]"
4,15156,"[O, O, O, O, O, B-LOCderiv, O, B-PER, I-PER, I-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Albert_Riera [2009-10-14],"[Nationalmannschaft, Natürlich, entging, auch, dem, spanischen, Nationaltrainer, Luis, Aragonés, Rieras, Leistungssteigerung, nicht, ,, so, dass, er, mittlerweile, drei, Spiele, für, die, "", Selección, "", bestritten, hat, .]"
5,12120,"[O, O, B-ORG, O, O, O, B-LOCderiv, O, O, O]","[O, O, O, O, O, O, O, O, O, O]",welt.de vom 15.02.2005 [2005-02-15],"[Damit, stattet, Walther, künftig, auch, die, irakischen, Sicherheitskräfte, aus, .]"
6,22008,"[O, O, O, O, O, O, O, O, B-PER, I-PER, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Zapp_(Magazin) [2009-11-28],"[Vom, 9., März, 2003, bis, November, 2005, führte, Caren, Miosga, durch, die, Sendung, .]"
7,22886,"[O, O, O, O, O, O, O, O, O, O, B-LOCderiv, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://www.handelsblatt.com/unternehmen/banken-versicherungen/ec-karten-debakel-banken-ueben-kritik-an-zka;2527576 [2010-02-11],"[Dem, Konsensprinzip, werde, "", in, Extenso, über, die, Säulen, des, deutschen, Bankensystems, gehuldigt, "", .]"
8,3155,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-OTH, I-OTH, O, O, B-OTH, I-OTH, I-OTH, I-OTH, I-OTH, I-OTH, O, B-PER, I-PER, I-PER, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Grosses_vollständiges_Universal-Lexicon_Aller_Wissenschafften_und_Künste [2009-12-29],"[Schon, der, erste, Artikel, zum, Buchstaben, „, A, “, sei, eine, Kompilation, aus, dem, Historischen, Lexicon, und, dem, Allgemeinen, Lexicon, der, Künste, und, Wissenschaften, von, Johann, Theodor, Jablonski, .]"
9,14850,"[B-PER, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, O]","[O, O, O, O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Nanaimo [2009-11-26],"[Dunsmuir, arbeitete, für, diese, Gesellschaft, und, für, die, Harewood, Coal, Company, .]"


## Preprocessing der Daten

Für das Preprocessign der Daten wird der Tokenizer des genutzen Modells benötigt. Dieser kann durch den folgenden Methodenaufruf heruntergeladen werden. <br>
`AutoTokenizer.from_pretrained()` lädt den zum Modell passenden Tokenizer sowie das Vokabular, welches für das Pretraining des Modelcheckpoints genutzt wurde, herunter.

In [13]:
from transformers import  AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast = True)

Der Tokenizer kann direkt genutzt werden, um eine Inputsequenz zu tokenisieren. Dabei erhält man ein Dictionary mit Mappings zu den input_ids, token_type_ids und eine attention_mask.
Die input_id ist die Identifikation des jeweiligen Tokens im Vokabluar des Modells.
Token_type_ids markieren Tokens in Seq2Seq Tasks und geben dem Modell Informationen darüber, zu welchem Teil einer zweiteiligen Eingabesequenz ein Token gehört.
Die attention_mask teilt dem Modell mit, für welche Token die Attention berechnet werden soll. Ist eine Eingabesequenz z. B. sehr kurz im Gegensatz zu den anderen, dann wird diese per Padding auf die gleiche Länge gebracht. Die attention_mask verhindert anschließend, dass die Attention für diese Padding Token berechnet wird.

Übergibt man dem Tokenizer nun eine String Sequenz, erhält man die oben beschriebene Ausgabe zurück:

In [14]:
tokenizer("Hallo, das ist ein Satz!")

{'input_ids': [101, 11763, 10133, 117, 10242, 10298, 10290, 61561, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Wenn im Datensatz, wie hier gegeben (Germeval_2014) die Inputs schon in einzelne Wörter gesplittet sind, dann kann man diese Liste und den Parameter `is_split_into_words=True` an den Tokenizer übergeben:

In [15]:
tokenizer(["Hallo",",","das", "ist", "ein","Satz","der","in","Wörter","aufgeteilt","wurde","!"],is_split_into_words=True)

{'input_ids': [101, 11763, 10133, 117, 10242, 10298, 10290, 61561, 10118, 10106, 160, 108036, 100261, 10283, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

#### Alignment- Problematik

Transformer Tokenizer nutzen meistens Subword Tokenizer, daher kann es vorkommen, dass selbst diese Wortliste noch in weitere Token zerlegt wird:

In [16]:
example = datasets["train"][5]
print(example["tokens"])

['ARD-Programmchef', 'Günter', 'Struve', 'war', 'wegen', 'eines', 'vierwöchigen', 'Urlaubs', 'für', 'eine', 'Stellungnahme', 'nicht', 'erreichbar', '.']


In [17]:
tokenized_input = tokenizer(example["tokens"],is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'ARD', '-', 'Programm', '##chef', 'Günter', 'St', '##ru', '##ve', 'war', 'wegen', 'eines', 'vier', '##w', '##ö', '##chi', '##gen', 'Ur', '##laub', '##s', 'für', 'eine', 'Stellung', '##nahme', 'nicht', 'erre', '##ich', '##bar', '.', '[SEP]']


Hier wurde zum Beispiel "Struve" in 3 Subtoken zerlegt und "vierwöchigen" sogar in 5 Subtoken.
Des Weiteren fällt auf, dass das hier verwendete Multilanguage BERT-Modell die Token in eine größere Anzahl von Subtokens zerlegt. Dies ist auch nachvollziehbar, da das Vokabular des Modells 104 Sprachen anstatt nur eine Sprache wie das deutsche BERT-Modell abdecken muss. Jeder Subtoken nimmt nämlich einen Platz im Vokabular des Modells ein.

Durch das Einfügen von SpecialToken [CLS] und [SEP] sowie die Subword Tokenisierung ist die Liste der Token länger als die Liste der zugehörigen Tags. Das Allignment aus dem Dataset ist somit kaputt:

In [18]:
len(example[f"{task}_tags"]),len(tokenized_input['input_ids'])

(14, 30)

Um dieses Problem zu lösen und das Alignment wiederherzustellen besitzt das Rückgabeobjektdes Tokenizer die `word_ids()` Methode.
- Sie liefert eine Liste die genauso lang ist, wie die Liste mit den Input-IDs
- Sie mapped Special Token zu `None` und alle anderen zum zugehörigen Original Wort-Input, bspw. markiert eine `0` die Zugehörigkeit des Subtokens zum ersten Token der Eingabesequenz

In [19]:
tokenized_input.word_ids()

[None,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 3,
 4,
 5,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 8,
 9,
 10,
 10,
 11,
 12,
 12,
 12,
 13,
 None]

Nun kann man das Alignment zwischen den `ner_tags` und den `input_ids` wiederherstellen. Im Ergbnis haben die Label und die Input_ids die gleiche Anzahl.

In [20]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

30 30


Mit der obigen Funktion wurden die `ner_tags` für die eingefügten Special Token  auf -100 gestellt und somit von Pytorch ignoriert. Die anderen `input_ids` haben das entsprechene Label ihres zugehörigen Wortes erhalten.

Eine andere Strategie ist es nur das Label für den ersten Token eines Wortes zu setzen und -100 für die weiteren Subtoken des Wortes zu vergeben. Dafür muss folgendes Flag geädnert werden:

In [21]:
label_all_tokens = True

#### Preprocessing Funktion

Mit der folgenden Funktion werden die übergebenen Datensätze des Datasets zunächst tokenisiert und anschließend werden die `ner_tags` mit den tokenisierten `input_ids` alligned. Dafür wird ein `labels` Attribut angelegt, welches diese Zuordnung enthält. Als Rückgabe erhält man die um das `labels` Attribut erweiterte Ausgabe des Tokenizers.

In [22]:
# Quelle https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=n9qywopnIrJH
def tokenize_and_align_labels(examples):
    # Tokenisierung der Eingabetoken
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special Tokens haben den Wert None als word id. Durch das Setzen des Wertes -100 als Label
            # wird dieser Token automatisch in der Loss Funktion ignoriert.
            if word_idx is None:
                label_ids.append(-100)
            # Setzen des Labels für den ersten Token eines Wortes.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # Abhängig vom label_all_tokems_flag wird für den nächsten Token eines Wortes das gleiche Label oder -100
            # gesetzt
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Die Funktion kann mit einem oder mehreren Examples genutzt werden. Wenn mehrere Datensätze übergeben werden, dann gibt der interne Tokenizer der Funktion eine  Liste mit Listen für jeden Key zurück:

In [23]:
tokenize_and_align_labels(datasets['train'][:3])

{'input_ids': [[101, 55260, 34567, 11705, 72721, 10268, 107, 98824, 54609, 87875, 107, 11036, 90928, 24603, 117, 19121, 13868, 107, 10106, 10599, 34375, 10329, 104667, 117, 10128, 25569, 12045, 10223, 10848, 60695, 10162, 10338, 107, 119, 102], [101, 36448, 66231, 58080, 17970, 10979, 38508, 13321, 17633, 17561, 10118, 10632, 10457, 102192, 11494, 10223, 150, 14902, 13770, 90409, 117, 10223, 10163, 10897, 58768, 69507, 10633, 12569, 29660, 10441, 10268, 49531, 54623, 119, 102], [101, 43019, 10632, 15800, 10496, 10268, 110575, 29033, 12044, 10392, 10328, 119, 10780, 10106, 11193, 10714, 10290, 184, 11013, 19115, 51699, 10107, 57716, 44837, 10790, 117, 10298, 17845, 10304, 13863, 61512, 79650, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Um die Funktion auf alle Datensätze im Datensatz anzuwenden kann man die `map` Methode des `datasets` Objekts nutzen. Dies wendet die Funktion auf alle Splits im `dataset` an, (training, valid, test set werden alle mit der einen Zeile Code preprocessed).
 - Durch `batched=True` wird dem Tokenizer ermöglicht mehrere Datensätze parallel zu verarbeiten

In [24]:
tokenized_dataset = datasets.map(tokenize_and_align_labels,batched=True)

Durch das Preprocessing werden die Ausgaben des Tokenizers als Features des DataSets ergänzt.
In den folgenden Ausgaben sieht man, dass nun die `attention_mask, die input_ids, die labels und die token_type_ids` als Features ergänzt wurden.

In [25]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'nested_ner_tags', 'source', 'token_type_ids', 'tokens'],
        num_rows: 24000
    })
    validation: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'nested_ner_tags', 'source', 'token_type_ids', 'tokens'],
        num_rows: 2200
    })
    test: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'ner_tags', 'nested_ner_tags', 'source', 'token_type_ids', 'tokens'],
        num_rows: 5100
    })
})

In [26]:
show_random_elements(tokenized_dataset["train"],seed=43,num_examples=2)

,attention_mask,id,input_ids,labels,ner_tags,nested_ner_tags,source,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1263,"[101, 10915, 34355, 10372, 56147, 12212, 10211, 13658, 11216, 23694, 10401, 10329, 119, 102]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, -100]","[O, O, O, O, O, O, B-LOC, O, O]","[O, O, O, O, O, O, O, O, O]",http://de.wikipedia.org/wiki/Khangchenne [2009-12-25],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Er, hielt, sich, lieber, im, fernen, Ngari, auf, .]"
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",9374,"[101, 57262, 25758, 10163, 14618, 144, 105687, 74456, 10441, 117, 10128, 29505, 11471, 10269, 63934, 69826, 34138, 10130, 10580, 41077, 10115, 10304, 85784, 119, 102]","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",http://www.verbaende.com/News.php4?m=56415 [2008-09-25],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Daher, lässt, er, keine, Gelegenheit, aus, ,, die, Politik, wachzurütteln, und, zum, Handeln, zu, bewegen, .]"


## Finetuning des Models

Für die Konstruktion des Modells wird ein Label-ID Mapping erzeugt. Dieses Mapping wird anschließend in der Configuration des Modells hinterlegt. Es ermöglicht dem Modell die vorhergesagten Label als Texte auszugeben anstatt ihrer numerischen Kodierung.

In [27]:
id2label = {}
label2id= {}
for i,l in enumerate(label_list):
    id2label[str(i)] = l
    label2id[l]=str(i)
print(id2label)
print(label2id)

{'0': 'O', '1': 'B-LOC', '2': 'I-LOC', '3': 'B-LOCderiv', '4': 'I-LOCderiv', '5': 'B-LOCpart', '6': 'I-LOCpart', '7': 'B-ORG', '8': 'I-ORG', '9': 'B-ORGderiv', '10': 'I-ORGderiv', '11': 'B-ORGpart', '12': 'I-ORGpart', '13': 'B-OTH', '14': 'I-OTH', '15': 'B-OTHderiv', '16': 'I-OTHderiv', '17': 'B-OTHpart', '18': 'I-OTHpart', '19': 'B-PER', '20': 'I-PER', '21': 'B-PERderiv', '22': 'I-PERderiv', '23': 'B-PERpart', '24': 'I-PERpart'}
{'O': '0', 'B-LOC': '1', 'I-LOC': '2', 'B-LOCderiv': '3', 'I-LOCderiv': '4', 'B-LOCpart': '5', 'I-LOCpart': '6', 'B-ORG': '7', 'I-ORG': '8', 'B-ORGderiv': '9', 'I-ORGderiv': '10', 'B-ORGpart': '11', 'I-ORGpart': '12', 'B-OTH': '13', 'I-OTH': '14', 'B-OTHderiv': '15', 'I-OTHderiv': '16', 'B-OTHpart': '17', 'I-OTHpart': '18', 'B-PER': '19', 'I-PER': '20', 'B-PERderiv': '21', 'I-PERderiv': '22', 'B-PERpart': '23', 'I-PERpart': '24'}


Erzeugen und Konfiguration des Modells mit Label-ID Mapping:
 - `AutoModelForTokenClassification.from_pretrained` lädt automatisch das entsprechende Modell herunter und initialisiert einen Token-Klassifizierungskopf am Ende des Modells.
 - Die auftretenden Warnungen geben nur Auskunft darüber, dass der Kopf des Modells ausgetauscht wurde und demzufolge keine trainierten Weights hat

In [28]:
from transformers import  AutoModelForTokenClassification, TrainingArguments, Trainer

In [29]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

#### Vorbereitung für die Erzeugung des `Trainers`:
Es müssen zunächst die `TrainingArguments` konfiguriert werden, dabei handelt es sich um die Hyperparameter des Trainings:
 - Dazu zählen z.B. die Anzahl der Epoch, die Learning Rate, die Batchsize und der weight_decay
 - `load_best_model_at_end` und `metric_for_best_model` sorgen dafür, dass am Ende des Trainings das Model mit der höchsten `F1_Score` geladen wird

In [30]:
metric_name = 'eval_f1'
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Der `DataCollator` fügt die Beispieldaten zu Batches zusammen und fügt das nötige Padding für die Inputs und Labels ein. (Hierfür wird die Länge des längsten Datensatzes gewählt)

In [31]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

Damit das Modell während des Trainings die gewünschten Metriken berechnen kann, muss eine Funktion definiert werden die diese Metriken berechnet. Das übernimmt die `compute_metric` Funktion:
- Die Datasets Bibliothek ermöglicht es, Funktionen zur Berechnung von Metriken herunterzuladen.
- `Seqeval` eignet sich gut für Tasks im Bereich der TokenClassification: https://github.com/chakki-works/seqeval

In [ ]:
#Installieren von seqeval falls nötig
#!pip install seqeval

Mit `load_metric` kann man eine gewünschte Funktion aus der Datasets Bibliothek herunterladen:

In [33]:
metric = load_metric("seqeval")

#### Beispiel für die Anwendung der Metrik

Um die Metriken zu berechnen, müssen der Funktion 2 Listen an die Parameter `predictions` und ` references` übergeben werden. 

Für dieses Beispiel wird die Labelliste für den `predictions` Parameter durch ein Mapping der `ner_tags` auf die Indizies der `label_list` erstellt:

In [34]:
labels = [label_list[i] for i in example[f"{task}_tags"]]

In [35]:
labels

['B-ORGpart',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

Für den `references` Parameter wird die Labelsliste leicht angepasst:

In [36]:
labels_ref = labels.copy()
labels_ref[-1] = 'B-ORGpart'

In [37]:
labels_ref

['B-ORGpart',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORGpart']

Anschließend kann die `compute` Methode des Metrikobjekts benutzt werden, um die Metrik zu berechnen:

In [38]:
metric.compute(predictions=[labels], references=[labels_ref])

{'ORGpart': {'f1': 0.6666666666666666,
  'number': 2,
  'precision': 1.0,
  'recall': 0.5},
 'PER': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 0.9285714285714286,
 'overall_f1': 0.8,
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666}

Damit diese Metrik vom Modell berechnet werden kann, muss sie in eine Funktion gewrappt werden.
Diese Funktion erhält während des Trainings das Ergebnis des Methodenaufrufs `Trainer.evaluate`. Dabei handelt es sich um ein Tuple aus den `predictions` und den tatsächlichen `labels`. Die `predictions` sind der letzte Hidden State des Modells, daher muss hiervon der maximale Wert ausgewählt werden, um tatsächlich vorhergesagten Wert zu erhalten. Die Funktion `compute_metrics` kümmert sich um das nötige Postprocessing und gibt die berechneten Metriken zurück.

In [39]:
import numpy as np

def compute_metrics(p):
    # Unpacking des Tuple p
    predictions, labels = p
    # Größten Wert für die jeweilige Prediction auswählen
    predictions = np.argmax(predictions, axis=2)

    # Ignorieren aller Werte mit -100
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

- hier wird die Ausgabe pro Kategorie weggelassen und nur Gesamt precision/recall/f1/accuracy  ausgegeben

Anschließend kann der `Trainer` erzeugt werden. Diesem wird das Modell, die Hyperparameter, die Trainings- und Evaluierungsdatensätze, der Tokenizer, der DataCollator sowie die `compute_metrics` Funktion übergeben.

In [40]:
trainer = Trainer(
    model,
    args,
    train_dataset =tokenized_dataset["train"],
    eval_dataset =tokenized_dataset["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Nun kann das Training mit der `train` Methode gestartet werden

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,0.216700,0.111921,0.782570,0.827593,0.804452,0.968283,9.963200,220.812000
2,0.084700,0.110632,0.799233,0.853845,0.825637,0.971318,9.930600,221.538000
3,0.056900,0.110794,0.816388,0.855148,0.835319,0.972199,9.915100,221.884000
4,0.037600,0.120922,0.816787,0.862409,0.838978,0.972851,10.006100,219.865000
5,0.028100,0.125785,0.820326,0.861106,0.840222,0.972819,9.974400,220.565000


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score a

TrainOutput(global_step=3750, training_loss=0.07851543896993002, metrics={'train_runtime': 1621.4975, 'train_samples_per_second': 2.313, 'total_flos': 7578051271734144.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 463412, 'init_mem_gpu_alloc_delta': 709410304, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1589805, 'train_mem_gpu_alloc_delta': 2845345792, 'train_mem_cpu_peaked_delta': 368051708, 'train_mem_gpu_peaked_delta': 1415178752})

Mit der evaluate Methode kann man das Model nochmals gegen das hinterlegte Dataset oder ein anderes Dataset validieren.

In [42]:
trainer.evaluate()

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 5.0,
 'eval_accuracy': 0.9728186386477844,
 'eval_f1': 0.8402216368425834,
 'eval_loss': 0.1257845163345337,
 'eval_mem_cpu_alloc_delta': 18120619,
 'eval_mem_cpu_peaked_delta': 8346732,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 95853056,
 'eval_precision': 0.8203263568641362,
 'eval_recall': 0.8611059393036679,
 'eval_runtime': 9.8131,
 'eval_samples_per_second': 224.19}

Evaluieren gegen das ungesehene Testset:

In [43]:
trainer.evaluate(tokenized_dataset["test"])

{'epoch': 5.0,
 'eval_accuracy': 0.9732980295739032,
 'eval_f1': 0.8287627551020408,
 'eval_loss': 0.12398409843444824,
 'eval_mem_cpu_alloc_delta': 41918012,
 'eval_mem_cpu_peaked_delta': 19282314,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 119267840,
 'eval_precision': 0.828630639247569,
 'eval_recall': 0.8288949130920108,
 'eval_runtime': 24.2285,
 'eval_samples_per_second': 210.496}

Im Anschluss kann man wieder die Precision/recall/f1/accuracy pro Kategorie berechnen indem man die `predict` Methode nutzt. Dazu wird die folgende `evaluate_all_categories` definiert. Sie erhält ein tokenisiertes Dataset als Eingabeparameter. Anschließend wird die Metrik für jede enthaltene Kategorie berechnet.

In [44]:
def evaluate_all_categories(tokenized_dataset):
    predictions, labels, _ = trainer.predict(tokenized_dataset)
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return metric.compute(predictions=true_predictions, references=true_labels)

In [45]:
evaluate_all_categories(tokenized_dataset["test"])

{'LOC': {'f1': 0.8949072711215778,
  'number': 3393,
  'precision': 0.8938547486033519,
  'recall': 0.8959622752726201},
 'LOCderiv': {'f1': 0.8772077375946173,
  'number': 1128,
  'precision': 0.8344,
  'recall': 0.924645390070922},
 'LOCpart': {'f1': 0.7221542227662179,
  'number': 429,
  'precision': 0.7603092783505154,
  'recall': 0.6876456876456877},
 'ORG': {'f1': 0.7761852260198456,
  'number': 2238,
  'precision': 0.7662168045276447,
  'recall': 0.7864164432529044},
 'ORGderiv': {'f1': 0.25,
  'number': 21,
  'precision': 1.0,
  'recall': 0.14285714285714285},
 'ORGpart': {'f1': 0.7387606318347509,
  'number': 801,
  'precision': 0.7195266272189349,
  'recall': 0.7590511860174781},
 'OTH': {'f1': 0.7045826513911619,
  'number': 1285,
  'precision': 0.7428817946505608,
  'recall': 0.6700389105058365},
 'OTHderiv': {'f1': 0.5419354838709678,
  'number': 75,
  'precision': 0.525,
  'recall': 0.56},
 'OTHpart': {'f1': 0.40117994100294985,
  'number': 182,
  'precision': 0.433121019

Das trainierte Modell schneidet besonders gut in den Hauptkategorien `LOC` und `PER` ab. Die nächstbesten Werte werden für die beiden anderen Hauptkategorien `ORG` und `OTH` erreicht. Dieses Ergebnis entspricht der Erwartung, da es für diese Kategorien auch die meisten Trainingsdaten gab. Besonders für die Klasse der `deriv` Token werden nicht so hohe F1-Werte erreicht. 

#### Speichern des trainierten Modells

In [46]:
trainer.save_model('models/deepset_finetuned/ner_multilang')

In [47]:
tokenizer.save_pretrained('models/deepset_finetuned/ner_multilang')

('models/deepset_finetuned/ner_multilang/tokenizer_config.json',
 'models/deepset_finetuned/ner_multilang/special_tokens_map.json',
 'models/deepset_finetuned/ner_multilang/vocab.txt',
 'models/deepset_finetuned/ner_multilang/added_tokens.json')

## Beispielhafte Anwendung des Modells

Um ein Transformers Modell einfach anzuwenden, nutzt man die Methode `pipeline`. Diese baut automatisch in Abhängigkeit vom übergebenen Task und Modell eine Pipeline. Die Pipeline kümmert sich um die nötigen Schritte um aus einer Stringeingabe eine Modellvorhersage zu erzeugen:

In [48]:
from transformers import pipeline

In [49]:
ner = pipeline("ner", model = f'models/deepset_finetuned/ner_multilang')

Some weights of BertModel were not initialized from the model checkpoint at models/deepset_finetuned/ner_multilang and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Anschließend kann dem `ner` Objekt der zu Verarbeitende String übergeben werden um eine Vorhersage der enthaltenden Entitäten zu erhalten:

In [50]:
ner('''In der CDU liegen die Nerven blank. Parteichef Laschet und andere führende Christdemokraten sprechen
    nach den Niederlagen in Mainz und Stuttgart von einem Warnschuss – und attackieren die SPD. ''')

[{'end': 10,
  'entity': 'B-ORG',
  'index': 3,
  'score': 0.9909046292304993,
  'start': 7,
  'word': 'CDU'},
 {'end': 50,
  'entity': 'B-PER',
  'index': 13,
  'score': 0.9976829886436462,
  'start': 47,
  'word': 'Las'},
 {'end': 54,
  'entity': 'B-PER',
  'index': 14,
  'score': 0.9975122213363647,
  'start': 50,
  'word': '##chet'},
 {'end': 134,
  'entity': 'B-LOC',
  'index': 28,
  'score': 0.9975348114967346,
  'start': 129,
  'word': 'Mainz'},
 {'end': 148,
  'entity': 'B-LOC',
  'index': 30,
  'score': 0.9977370500564575,
  'start': 139,
  'word': 'Stuttgart'},
 {'end': 195,
  'entity': 'B-ORG',
  'index': 42,
  'score': 0.9937661290168762,
  'start': 192,
  'word': 'SPD'}]

Das Modell konnte für obige Eingabe alle Token korrekt bestimmen.

In [51]:
ner('''Die Europäische Union hat ein Verfahren gegen Großbritannien wegen Verletzung des EU-Austrittsvertrags
    eingeleitet. Dies teilte die EU-Kommission am Montag in Brüssel mit. ''')

[{'end': 15,
  'entity': 'B-ORG',
  'index': 2,
  'score': 0.9944987297058105,
  'start': 4,
  'word': 'Europäische'},
 {'end': 21,
  'entity': 'I-ORG',
  'index': 3,
  'score': 0.9931507706642151,
  'start': 16,
  'word': 'Union'},
 {'end': 60,
  'entity': 'B-LOC',
  'index': 8,
  'score': 0.9960834980010986,
  'start': 46,
  'word': 'Großbritannien'},
 {'end': 84,
  'entity': 'B-ORGpart',
  'index': 13,
  'score': 0.9781320095062256,
  'start': 82,
  'word': 'EU'},
 {'end': 85,
  'entity': 'B-ORGpart',
  'index': 14,
  'score': 0.9844021797180176,
  'start': 84,
  'word': '-'},
 {'end': 89,
  'entity': 'B-ORGpart',
  'index': 15,
  'score': 0.9874082803726196,
  'start': 85,
  'word': 'Aust'},
 {'end': 93,
  'entity': 'B-ORGpart',
  'index': 16,
  'score': 0.9885257482528687,
  'start': 89,
  'word': '##ritt'},
 {'end': 94,
  'entity': 'B-ORGpart',
  'index': 17,
  'score': 0.9874943494796753,
  'start': 93,
  'word': '##s'},
 {'end': 101,
  'entity': 'B-ORGpart',
  'index': 18,
  's

In [52]:
ner('''Das sogenannte Nordirland-Protokoll im Austrittsvertrag sieht vor,
    dass einige Regeln des EU-Binnenmarkts für Nordirland weiter gelten.
    Dies soll Kontrollen an der Landgrenze zum EU-Staat Irland auf der gemeinsamen
    Insel überflüssig machen. Da Waren dennoch kontrolliert werden müssen, um EU-Standards zu wahren,
    wurden die Kontrollen auf Häfen an der Irischen See zwischen Nordirland und dem übrigen
    Großbritannien verschoben. So wurde das Problem zwischen Großbritannien und der
    EU – und insbesondere der europäischen Republik Irland – zu einem innerbritischen Problem.''')

[{'end': 19,
  'entity': 'B-LOCpart',
  'index': 3,
  'score': 0.8855133652687073,
  'start': 15,
  'word': 'Nord'},
 {'end': 21,
  'entity': 'B-LOCpart',
  'index': 4,
  'score': 0.924804151058197,
  'start': 19,
  'word': '##ir'},
 {'end': 25,
  'entity': 'B-LOCpart',
  'index': 5,
  'score': 0.9399065375328064,
  'start': 21,
  'word': '##land'},
 {'end': 26,
  'entity': 'B-LOCpart',
  'index': 6,
  'score': 0.9164689183235168,
  'start': 25,
  'word': '-'},
 {'end': 31,
  'entity': 'B-LOCpart',
  'index': 7,
  'score': 0.8861373066902161,
  'start': 26,
  'word': 'Proto'},
 {'end': 34,
  'entity': 'B-LOCpart',
  'index': 8,
  'score': 0.8672819137573242,
  'start': 31,
  'word': '##kol'},
 {'end': 35,
  'entity': 'B-LOCpart',
  'index': 9,
  'score': 0.8002591729164124,
  'start': 34,
  'word': '##l'},
 {'end': 43,
  'entity': 'B-OTH',
  'index': 11,
  'score': 0.5414198637008667,
  'start': 39,
  'word': 'Aust'},
 {'end': 47,
  'entity': 'B-OTH',
  'index': 12,
  'score': 0.484786

In [53]:
ner('''@_A_K_K_ @CDU @jensspahn @_FriedrichMerz Die CDU muss für den packt und die betrügerische
    Wahl karrenbauer muss absteigen auf 12% Überall sind die Regierungen gegen den packt Deutschland
    wird von irren in den Abgrund gerissen.''')

[{'end': 13,
  'entity': 'B-ORG',
  'index': 10,
  'score': 0.9232537150382996,
  'start': 10,
  'word': 'CDU'},
 {'end': 20,
  'entity': 'B-LOC',
  'index': 13,
  'score': 0.7682573795318604,
  'start': 18,
  'word': '##ss'},
 {'end': 24,
  'entity': 'B-LOC',
  'index': 15,
  'score': 0.7890185117721558,
  'start': 23,
  'word': '##n'},
 {'end': 36,
  'entity': 'B-LOC',
  'index': 18,
  'score': 0.9671655297279358,
  'start': 27,
  'word': 'Friedrich'},
 {'end': 37,
  'entity': 'B-LOC',
  'index': 19,
  'score': 0.9699755907058716,
  'start': 36,
  'word': '##M'},
 {'end': 40,
  'entity': 'B-LOC',
  'index': 20,
  'score': 0.9771344065666199,
  'start': 37,
  'word': '##erz'},
 {'end': 48,
  'entity': 'B-ORG',
  'index': 22,
  'score': 0.9460809230804443,
  'start': 45,
  'word': 'CDU'},
 {'end': 102,
  'entity': 'B-PER',
  'index': 35,
  'score': 0.5276461243629456,
  'start': 99,
  'word': 'kar'},
 {'end': 105,
  'entity': 'B-PER',
  'index': 36,
  'score': 0.4812789857387543,
  'st

Das Modell erkennt anhand der obigen Eingabe auch Personenentitäten in Tweets wie bspw. @jensspahn und @_FriedrichMerz